 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [2]:
# Reading whale returns: 
#Step 1: Calling the original file path 
# aapl = Path("UCB_FinTech/homework_2/Resources/aapl_historical.csv")
# goog = Path("UCB_FinTech/homework_2/Resources/goog_historical.csv")
# cost = Path("UCB_FinTech/homework_2/Resources/cost_historical.csv")

#Step 2: Reading the CSV
# aapl = pd.read_csv("./Resources/aapl_historical.csv")
# goog = pd.read_csv("./Resources/goog_historical.csv")
# cost = pd.read_csv("./Resources/cost_historical.csv")
whale = pd.read_csv("./Resources/whale_returns.csv")

#Step 3: Testing that file called are valid (#commented out) 
# whale.head()

In [3]:
# Count nulls
whale_nulls = whale.isnull().sum().sum()
print(whale_nulls)

4


In [4]:
# Drop nulls
whale = whale.dropna()

#checking the nulls: 
# whale_nulls = whale.isnull().sum().sum()
# print(whale_nulls)
#Check cleared, there are no 0 nulls

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [5]:
# Reading algorithmic returns
# algo = Path("UCB_FinTech/homework_2/Resources/algo_returns.csv")
algo = pd.read_csv("./Resources/algo_returns.csv")


In [6]:
# Count nulls
algo_nulls = algo.isnull().sum().sum() #Sums the data frame to a 2nd degree. (sum of sum)
print(algo_nulls)

6


In [7]:
# Drop nulls
algo = algo.dropna()
# print(algo.isnull().sum().sum())

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [8]:
# Reading S&P 500 Closing Prices
spy = Path("UCB_FinTech/homework_2/Resources/sp500_history.csv")
spy = pd.read_csv("./Resources/sp500_history.csv")
spy.head()

,Date,Close
0,23-Apr-19,$2933.68
1,22-Apr-19,$2907.97
2,18-Apr-19,$2905.03
3,17-Apr-19,$2900.45
4,16-Apr-19,$2907.06


In [9]:
# Check Data Types
print(spy.dtypes) #checked, it is object 


Date     object
Close    object
dtype: object


In [19]:
# Fix Data Types
# print(spy.isnull().sum().sum()) # no nulls
# examined situation: Date = Object, this is good. 
#However: Close prices = "$" and are object
spy.head()

#solution: Convert the "Close" price as a "Str" 
spy['Close'] = spy['Close'].astype("str") 


#step 2: Stripped the "$" 

# print(type(spy['Close']))
# spy['Close'] = spy['Close'].strip("$")
spy['Close'] = spy['Close'].str.lstrip("$").astype("float")

#step 3: Cast the pandas object to dtype: Float 
# print(spy)
# spy['Close'] = spy['Close'].astype("float")
print(spy)


           Date    Close
0     23-Apr-19  2933.68
1     22-Apr-19  2907.97
2     18-Apr-19  2905.03
3     17-Apr-19  2900.45
4     16-Apr-19  2907.06
...         ...      ...
1644   5-Oct-12  1460.93
1645   4-Oct-12  1461.40
1646   3-Oct-12  1450.99
1647   2-Oct-12  1445.75
1648   1-Oct-12  1444.49

[1649 rows x 2 columns]


In [28]:
# Calculate Daily Returns
print(spy['Close'].pct_change())

perc_change = spy['Close'].pct_change()
# spy.combine(perc_change)
# print(
# type(perc_change)

spy['Close'] = perc_change

0            NaN
1      -0.008764
2      -0.001011
3      -0.001577
4       0.002279
          ...   
1644    0.003469
1645    0.000322
1646   -0.007123
1647   -0.003611
1648   -0.000872
Name: Close, Length: 1649, dtype: float64


In [32]:
# Drop nulls
spy = spy.dropna()
# print(spy)/

           Date     Close
1     22-Apr-19 -0.008764
2     18-Apr-19 -0.001011
3     17-Apr-19 -0.001577
4     16-Apr-19  0.002279
5     15-Apr-19 -0.000509
...         ...       ...
1644   5-Oct-12  0.003469
1645   4-Oct-12  0.000322
1646   3-Oct-12 -0.007123
1647   2-Oct-12 -0.003611
1648   1-Oct-12 -0.000872

[1648 rows x 2 columns]


In [36]:
# Rename `Close` Column to be specific to this portfolio.
# spy = spy['Close'].rename(columns = "Percent Change")

spy = spy.rename(columns = {"Close":"Percent Change"})
print(spy)

           Date  Percent Change
1     22-Apr-19       -0.008764
2     18-Apr-19       -0.001011
3     17-Apr-19       -0.001577
4     16-Apr-19        0.002279
5     15-Apr-19       -0.000509
...         ...             ...
1644   5-Oct-12        0.003469
1645   4-Oct-12        0.000322
1646   3-Oct-12       -0.007123
1647   2-Oct-12       -0.003611
1648   1-Oct-12       -0.000872

[1648 rows x 2 columns]


## Combine Whale, Algorithmic, and S&P 500 Returns

In [37]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.

#Steps: 
#Index each dataframe using the date 
#Use 'Concat' on the dataframes to combine them 

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [15]:
# Plot daily returns of all portfolios


#### Calculate and Plot cumulative returns.

In [16]:
# Calculate cumulative returns of all portfolios

# Plot cumulative returns


---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [17]:
# Box plot to visually show risk


### Calculate Standard Deviations

In [18]:
# Calculate the daily standard deviations of all portfolios


### Determine which portfolios are riskier than the S&P 500

In [19]:
# Calculate  the daily standard deviation of S&P 500

# Determine which portfolios are riskier than the S&P 500


### Calculate the Annualized Standard Deviation

In [20]:
# Calculate the annualized standard deviation (252 trading days)


---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [21]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window

# Plot the rolling standard deviation


### Calculate and plot the correlation

In [22]:
# Calculate the correlation

# Display de correlation matrix


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [23]:
# Calculate covariance of a single portfolio

# Calculate variance of S&P 500

# Computing beta

# Plot beta trend


## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [24]:
# Use `ewm` to calculate the rolling window


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [25]:
# Annualized Sharpe Ratios


In [26]:
# Visualize the sharpe ratios as a bar plot


### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [27]:
# Reading data from 1st stock


In [28]:
# Reading data from 2nd stock


In [29]:
# Reading data from 3rd stock


In [30]:
# Combine all stocks in a single DataFrame


In [31]:
# Reset Date index


In [32]:
# Reorganize portfolio data by having a column per symbol


In [33]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [34]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [35]:
# Join your returns DataFrame to the original returns DataFrame


In [36]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [37]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [38]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [39]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [40]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [41]:
# Calculate Annualized Sharpe Ratios


In [42]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!